<a href="https://colab.research.google.com/github/ermolushka/trading-gpt/blob/master/ticker_list_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Filter out companies in the constituent of the index

In [66]:
import requests
import pandas as pd
EOD_API_KEY = "5e972a7685a114.008405276404"
FMP_API_KEY = 'a247dff6e3a9a1dbda83cd362187b280'
index_name = "RUT.INDX"

In [67]:
def get_eod_index_constituent_list(index_name):
  url = f"https://eodhistoricaldata.com/api/fundamentals/{index_name}?api_token={EOD_API_KEY}"

  r = requests.get(url)

  if r.status_code != 200:
    return None
  else:
    data = r.json()
  return data

In [68]:
index_constitutent_df = pd.DataFrame(get_eod_index_constituent_list(index_name)['Components']).transpose()
index_constitutent_df

,Code,Exchange,Name,Sector,Industry
0,VABK,US,Virginia National Bankshares Corp,Financial Services,Banks-Regional
1,NEWT,US,Newtek Business Services Corp,Financial Services,Banks-Regional
2,HEAR,US,Turtle Beach Corp,Technology,Consumer Electronics
3,CPE,US,Callon Petroleum Company,Energy,Oil & Gas E&P
4,PBH,US,Prestige Brand Holdings Inc,Healthcare,Drug Manufacturers-Specialty & Generic
...,...,...,...,...,...
1990,KNF,US,Knife River Corporation,Basic Materials,Building Materials
1991,OBK,US,Origin Bancorp Inc.,Financial Services,Banks-Regional
1992,METCV,US,Ramaco Resources Inc. Class A Common Stock Ex-...,Basic Materials,Coking Coal
1993,WKC,US,World Kinect Corporation,Energy,Oil & Gas Refining & Marketing


In [74]:
filtered_index_df = index_constitutent_df[index_constitutent_df['Industry'] == 'Biotechnology']
filtered_index_df

,Code,Exchange,Name,Sector,Industry
19,INSM,US,Insmed Inc,Healthcare,Biotechnology
29,ADMA,US,ADMA Biologics Inc,Healthcare,Biotechnology
49,AVXL,US,Anavex Life Sciences Corp,Healthcare,Biotechnology
103,DYN,US,Dyne Therapeutics Inc,Healthcare,Biotechnology
125,ARWR,US,Arrowhead Pharmaceuticals Inc,Healthcare,Biotechnology
...,...,...,...,...,...
1973,MLYS,US,Mineralys Therapeutics Inc. Common Stock,Healthcare,Biotechnology
1977,ELVN,US,Enliven Therapeutics Inc.,Healthcare,Biotechnology
1980,ZVRA,US,Zevra Therapeutics Inc.,Healthcare,Biotechnology
1982,CARM,US,Carisma Therapeutics Inc.,Healthcare,Biotechnology


In [75]:
 filtered_index_df.to_csv(f'{index_name}_filtered_ticker_list.csv')

##Filter out in the list the companies that don't publish their news on globenewswire

In [76]:
def get_fmp_news_stock(ticker):
  url = f'https://financialmodelingprep.com/api/v3/stock_news?tickers={ticker}&limit=1000&apikey={FMP_API_KEY}'

  r = requests.get(url)
  if r.status_code != 200:
    return None
  else:
    data = r.json()

  return data


def get_only_globenewswire(ticker):
  data = get_fmp_news_stock(ticker=ticker)
  globe_wire_ticker_list = [elem for elem in data if elem['site'] == 'GlobeNewsWire']

  if len(globe_wire_ticker_list) > 0:
    return True
  else:
    return False


In [82]:
tmp_df = filtered_index_df.copy()
tmp_df['is_globe_news_wire'] = tmp_df['Code'].map(lambda x: get_only_globenewswire(x))
filtered_globe_wire = tmp_df[tmp_df['is_globe_news_wire'] == True]
filtered_globe_wire

,Code,Exchange,Name,Sector,Industry,is_globe_news_wire
29,ADMA,US,ADMA Biologics Inc,Healthcare,Biotechnology,True
49,AVXL,US,Anavex Life Sciences Corp,Healthcare,Biotechnology,True
103,DYN,US,Dyne Therapeutics Inc,Healthcare,Biotechnology,True
125,ARWR,US,Arrowhead Pharmaceuticals Inc,Healthcare,Biotechnology,True
321,FOLD,US,Amicus Therapeutics Inc,Healthcare,Biotechnology,True
...,...,...,...,...,...,...
1956,PRME,US,Prime Medicine Inc. Common Stock,Healthcare,Biotechnology,True
1966,IRON,US,Disc Medicine Inc.,Healthcare,Biotechnology,True
1973,MLYS,US,Mineralys Therapeutics Inc. Common Stock,Healthcare,Biotechnology,True
1977,ELVN,US,Enliven Therapeutics Inc.,Healthcare,Biotechnology,True


In [84]:
filtered_globe_wire.to_csv(f'globe_news_wire_filtered_tickers.csv')